# 입문자를 위한, 파이썬/R 데이터 분석  

## Part(1) - Day06  : 

]

## Today's mission

- (크롤링) 시카고 샌드위치 맛집 소개 사이트에 접근하기

]

## (크롤링) 시카고 샌드위치 맛집 소개 사이트에 접근하기

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [ ]:
url_base = "https://www.chicagomag.com"
url_sub = "/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/"
url = url_base + url_sub

In [ ]:
url

In [ ]:
import requests
from bs4 import BeautifulSoup
#res = requests.get(url)
headers = {'User-Agent': 'Mozilla/5.0'}
res = requests.get(url, headers=headers)
res

In [ ]:
soup = BeautifulSoup(res.text)
print(soup)

In [ ]:
print(soup.find_all('div', 'sammy'))

In [ ]:
len(soup.find_all('div', 'sammy'))

In [ ]:
print(soup.find_all('div', 'sammy')[0])

### 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [ ]:
tmp_one = soup.find_all('div', 'sammy')[0]
type(tmp_one)

In [ ]:
tmp_one.find(class_='sammyRank').get_text()

In [ ]:
tmp_one.find(class_='sammyListing').get_text()

In [ ]:
tmp_one.find('a')['href']

In [ ]:
import re
tmp_string = tmp_one.find(class_='sammyListing').get_text()
tmp_string#.split()

In [ ]:
re.split(('\n|\r\n'), tmp_string)

In [ ]:
print(re.split(('\n|\r\n'), tmp_string)[0])
print(re.split(('\n|\r\n'), tmp_string)[1])

In [ ]:
from urllib.request import urljoin

In [ ]:
rank = []
main_menu = []
cafe_name = []
url_add = []

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_="sammyRank").get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

In [ ]:
rank[:5]

In [ ]:
main_menu[:5]

In [ ]:
cafe_name[:5]

In [ ]:
url_add[:5]

In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

In [ ]:
import pandas as pd

In [ ]:
data = {'Rank':rank, 'Cafe':cafe_name, 'Menu':main_menu, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

In [ ]:
df.to_csv('data/best_sandwiches_list_chicago_raw.csv', sep=",", encoding='UTF-8')

### 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [ ]:
df = pd.read_csv('data/best_sandwiches_list_chicago_raw.csv', index_col=0)
df.head()

In [ ]:
df['URL'][0]

In [ ]:
html = df['URL'][0]
headers = {'User-Agent': 'Mozilla/5.0'}
res = requests.get(html, headers=headers)
soup_tmp = BeautifulSoup(res.text)
print(soup_tmp)

In [ ]:
print(soup_tmp.find('p', 'addy'))

In [ ]:
price_tmp = soup_tmp.find('p', 'addy').get_text()
print(price_tmp)

In [ ]:
price_tmp.split()

In [ ]:
price_tmp.split()[0]

In [ ]:
price_tmp.split()[0][:-1]

In [ ]:
price_tmp.split()[1]

In [ ]:
price_tmp.split()[-2]

In [ ]:
' '.join(price_tmp.split()[1:-2])[:-1]

In [ ]:
df.index[:3]

In [ ]:
price = []
address = []

for n in df.index[:3]:
    html = df['URL'][n]
    res = requests.get(html, headers=headers)
    soup_tmp = BeautifulSoup(res.text)
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2])[:-1])

In [ ]:
price

In [ ]:
address

### 상태 진행바 적용 후 샌드위치 페이지 50개 접근하기

In [ ]:
#!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
import time
 
for i in tqdm(range(10)): ## tqdm 첫 번째 인자에 순회가능한 객체 전달
    time.sleep(0.01)

In [ ]:
price = []
address = []

for n in tqdm(df.index):
    html = df['URL'][n]
    res = requests.get(html, headers=headers)
    soup_tmp = BeautifulSoup(res.text)
    
    gettings = soup_tmp.find('p', 'addy').get_text()
    price.append(gettings.split()[0][:-1])
    address.append(' '.join(gettings.split()[1:-2])[:-1])

### 50개 웹 페이지에 대한 정보 가져오기

In [ ]:
price

In [ ]:
address

In [ ]:
len(price), len(address), len(df)

In [ ]:
df['Price'] = price
df['Address'] = address
df

In [ ]:
#df[['Rank', 'Cafe', 'Menu', 'Price', 'Address']]

In [ ]:
df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head()

In [ ]:
df.to_csv('data/best_sandwiches_list_chicago_result.csv', sep=',', encoding='UTF-8')